## Import Data From Snowflake  
In this section we will import the data from Snowflake for profiling and univariate analyses

In [2]:
# Requried libraries for importing
import pandas as pd
import numpy as np
import snowflake.connector as snow
from snow_conn      import snow_cred
from sqlalchemy     import create_engine
from sqlalchemy.sql import text
from snow_conn_yluo import snow_cred
# added a comment
# and another comment

In [3]:
# Create the Snowflake connection engine based on the desired settings
account_identifier = 'axcessfinancial.us-east-1'
user           = snow_cred.username
password       = snow_cred.password
warehouse_name = 'ANALYTICS_XS_WH' #### changet the warehouse if speed is too low
role_name      = 'YLUO_ROLE'

conn_string = f"snowflake://{user}:{password}@{account_identifier}/?warehouse={warehouse_name}&role={role_name}"
engine      = create_engine(conn_string)
connection  = engine.connect()

sql = """select *
         from EDW_RPTG_DB.CREDIT.XACT_RTUGEN2_MODELS_AND_KSDEDINALS_NODUP_UPDATE_RAW
      """

with engine.connect() as conn:
    query = conn.execute(text(sql))

try:
    db_0 = pd.DataFrame(query.fetchall())
finally:
    connection.close()
    engine.dispose()

In [6]:
from datetime import datetime as dt
# Prepare a base data

# Keep only not funded apps for 30DPD in 9M KPI and loan date range from Jan, 2021 to April, 2022
db_1 = db_0.loc[db_0['booked_flag'] == 0]
db_1['app_date'] = pd.to_datetime(db_1['app_date'], format='%Y-%m-%d')

# set the start and end dates
start_date = pd.Timestamp('2021-01-01')
end_date = pd.Timestamp('2022-04-30')

# filter the DataFrame based on the date range
db_1 = db_1.loc[(db_1['app_date'] >= start_date) & (db_1['app_date'] <= end_date)]

In [7]:
# Variable list for columnns from original dataset
vlist = [x.lower() for x in ['APP_STATE', 'uw_customer_type', 'app_date', 'pre_uw_denial_flag', 'TOTAL_MONTHLY_NET_INCOME', 'TOTAL_MONTHLY_GROSS_INCOME', 'BANK_BAL', 'BANK_ACCTOPENDAYS', '_30DPD_9M_FLAG']]

# Variable list for all the attributes and generic scores used in the TUGEN2 RCLP Scorecards (used for Xact product)
vlist = vlist + [x.lower() for x in ['AGG905','AGG907','AT01S','AT104S','AT32S','ATAP01','AU34S','AUT222','BALMAG01','BKC102','CVPL','FI06S','FI20S','FI24S','FI28S','G102S','G207S','G404S','LINKC033','LINKF036','LINKF063','LINKF084','LINKF106','LINKF109','LINKF111','LINKF148','LINKF151','LINKF182','LINKF185','LINKF189','LINKF190','M1150_STL_FT_TU','M1151_INS_FT_TU','MTAP01','PAYMNT11','PER233','RET225','REV205','S064A','SCORE_CV_EARLYPAYDEFAULT','SCORE_CV_NEWACCT_2','SCR_ALT_B_2','SCR_LINK_B_2','SE09S','SE20S','SE34S','VANTAGESCORE4']]

db_2 = db_1.loc[:,vlist].copy()

In [8]:
# Re-format to numerics

# Get a list of all of the columns needed to be re-formatted
vlist_cat = ['bank_bal'] + [x.lower() for x in ['AGG905','AGG907','AT01S','AT104S','AT32S','ATAP01','AU34S','AUT222','BALMAG01','BKC102','CVPL','FI06S','FI20S','FI24S','FI28S','G102S','G207S','G404S','LINKC033','LINKF036','LINKF063','LINKF084','LINKF106','LINKF109','LINKF111','LINKF148','LINKF151','LINKF182','LINKF185','LINKF189','LINKF190','M1150_STL_FT_TU','M1151_INS_FT_TU','MTAP01','PAYMNT11','PER233','RET225','REV205','S064A','SCORE_CV_EARLYPAYDEFAULT','SCORE_CV_NEWACCT_2','SCR_ALT_B_2','SCR_LINK_B_2','SE09S','SE20S','SE34S','VANTAGESCORE4']]

db_3 = db_2.copy()
# Iterate through the columns, converting them to numeric values
for col in vlist_cat:
    db_3[col] = pd.to_numeric(db_3[col])

# Check data types for all columns
# db_3.dtypes

In [9]:
import os

# set working directory where the output files will be saved
os.chdir('C:/Users/yluo.CNG/OneDrive - Axcess Financial Services, Inc/Documents/_local_repo/analytics-xact-tugen2-refit/data')
print(os.getcwd())

C:\Users\yluo.CNG\OneDrive - Axcess Financial Services, Inc\Documents\_local_repo\analytics-xact-tugen2-refit\data


In [10]:
db_4 = db_3.copy().drop('linkf190',axis=1)

vlist_small_missing = [
    'agg905','agg907','at01s','at104s','at32s','atap01','au34s','aut222','balmag01','bkc102','cvpl','fi06s','fi20s','fi24s','fi28s',
    'g102s','g207s','g404s','mtap01','paymnt11','per233','ret225','rev205','s064a','score_cv_earlypaydefault','score_cv_newacct_2',
    'scr_alt_b_2','scr_link_b_2','se09s','se20s','se34s','vantagescore4'
]
# modify the missing values in order to group them into the other bins in the later steps
for ivars in vlist_small_missing:
    db_4.loc[db_4[ivars].isnull(), ivars] = -9999


In [11]:
# get the variables and the corresponding special values
tb_small_special = {
    'agg905': [-2, -6],
    'agg907': [-2, -6],
    'at104s': [-4],
    'at32s': [-2, -3, -4],
    'atap01': [-2, -3, -4],
    'au34s': [-2, -4],
    'aut222': [-6],
    'balmag01': [-1, -2, -6],
    'bkc102': [-6],
    'fi06s': [-4, -5],
    'fi20s': [-4, -5],
    'fi24s': [-4],
    'fi28s': [-2, -4],
    'g102s': [-4],
    'g207s': [-4, -5],
    'g404s': [-4],
    'mtap01': [-2, -4],
    'paymnt11': [-2, -6],
    'per233': [-6],
    'ret225': [-6],
    'rev205': [-1, -6],
    'se09s': [-4],
    'se20s': [-4],
    'se34s': [-2, -4, -5]
}

for variable, values_to_replace in tb_small_special.items():
    if variable in db_4.columns:
        db_4[variable] = db_4[variable].replace(values_to_replace, -9999)

In [12]:
db_5 = db_4.copy()

db_5.loc[db_5['agg905'] == -9999, 'agg905'] = 105
db_5.loc[db_5['agg907'] == -9999, 'agg907'] = 2704
db_5.loc[db_5['at01s'] == -9999, 'at01s'] = 17
db_5.loc[db_5['at32s'] == -9999, 'at32s'] = 932
db_5.loc[db_5['at104s'] == -9999, 'at104s'] = 93
db_5.loc[db_5['atap01'] == -9999, 'atap01'] = 2908
db_5.loc[db_5['au34s'] == -1, 'au34s'] = -9999
db_5.loc[db_5['balmag01'] == -9999, 'balmag01'] = 377
db_5.loc[db_5['cvpl'] == -9999, 'cvpl'] = 490
db_5.loc[db_5['fi06s'] == -1, 'fi06s'] = -9999
db_5.loc[db_5['fi20s'] == -1, 'fi20s'] = -9999
db_5.loc[db_5['fi28s'] == -1, 'fi28s'] = -9999
db_5.loc[db_5['g102s'] == -9999, 'g102s'] = 1
db_5.loc[db_5['g207s'] == -9999, 'g207s'] = 80
db_5.loc[db_5['g404s'] == -9999, 'g404s'] = 1
db_5.loc[db_5['mtap01'] == -1, 'mtap01'] = -9999
db_5.loc[db_5['paymnt11'] == -9999, 'paymnt11'] = 4
db_5.loc[db_5['s064a'] == -4, 's064a'] = -9999
db_5.loc[db_5['score_cv_earlypaydefault'] == -9999, 'score_cv_earlypaydefault'] = 570
db_5.loc[db_5['score_cv_newacct_2'] == -9999, 'score_cv_newacct_2'] = 575
db_5.loc[db_5['scr_alt_b_2'] == -9999, 'scr_alt_b_2'] = 578
db_5.loc[db_5['scr_link_b_2'] == -9999, 'scr_link_b_2'] = 591
db_5.loc[db_5['se09s'] == -9999, 'se09s'] = 3
db_5.loc[db_5['se20s'] == -9999, 'se20s'] = 10
db_5.loc[db_5['se34s'] == -9999, 'se34s'] = 92
db_5.loc[db_5['vantagescore4'] == -9999, 'vantagescore4'] = 581

In [13]:
db_bin = pd.read_csv('optbinning_result_bins(final adjusted).csv')

In [14]:
ref_table = {
    'app_state': {
        'MI': 0.169985208,
        'IN': 0.158817868,
        'OH': 0.09431183,
        'TX': 0.083342484,
        'FL': -0.12014968,
        'TN': -0.264285899,
        'other': -0.002663082,
    },
    'total_monthly_net_income': {
        (-np.inf, 1429.00): -0.300251048,
        (1429.00, 1884.50): -0.242433391,
        (1884.50, 2332.50): -0.118723299,
        (2332.50, 3135.00): -0.003449182,
        (3135.00, 3303.00): 0.007352002,
        (3303.00, 3572.50): 0.029547597,
        (3572.50, 4399.00): 0.14388839,
        (4399.00, 6887.00): 0.168950447,
        (6887.00, np.inf): 0.359583415,
        'other': -0.300251048
    },
    'total_monthly_gross_income': {
        (-np.inf, 1413.50):-0.343530568,
        (1413.50, 2305.50):-0.296943689,
        (2305.50, 3145.50):-0.154988498,
        (3145.50, 3327.00):-0.060325167,
        (3327.00, 4013.50):-0.00628043,
        (4013.50, 4562.00):0.061392092,
        (4562.00, 6112.50):0.17826932,
        (6112.50, 8130.50):0.263200408,
        (8130.50, np.inf):0.322739708,
        'other':-0.343530568
    },
    'bank_bal': {
        (-np.inf, 1.50):-0.492167548,
        (1.50, 7.50):-0.447005624,
        (7.50, 21.50):-0.222825131,
        (21.50, 53.50):-0.071575257,
        (53.50, 125.50):0.014649012,
        (125.50, 216.50):0.113819691,
        (216.50, 853.50):0.171756182,
        (853.50, 1523.50):0.329621152,
        (1523.50, np.inf):0.373243426,
        'other':-0.492167548
    },
    'bank_acctopendays': {
        (-np.inf, 428.50):-0.481663625,
        (428.50, 646.50):-0.406785962,
        (646.50, 1020.50):-0.290310853,
        (1020.50, 1393.50):-0.167168759,
        (1393.50, 1623.50):-0.087952766,
        (1623.50, 2330.50):0.082109114,
        (2330.50, 3368.50):0.218069768,
        (3368.50, 4754.00):0.220313614,
        (4754.00, 6933.50):0.339630148,
        (6933.50, np.inf):0.452764202,
        'other':-0.481663625
    },
    'agg905': {
        (0, 34.01):-0.099293219,
        (34.01, 59.46):0.12484279,
        (59.46, 69.52):0.222352499,
        (69.52, 97.92):0.094910008,
        (97.92, 104.88):0.070637799,
        (104.88, 121.55):-0.116262978,
        (121.55, np.inf):-0.397250559,
        (-1, -0.99):-0.269949948,
        'other':-0.099293219
    },
    'agg907': {
        (0, 309.50):-0.281391163,
        (309.50, 697.50):-0.240654295,
        (697.50, 1223.50):-0.103096411,
        (1223.50, 1719.50):-0.091479162,
        (1719.50, 2546.50):0.014057201,
        (2546.50, 2704.50):0.019603054,
        (2704.50, 4016.50):0.100054783,
        (4016.50, 5868.00):0.15715062,
        (5868.00, 12494.50):0.237902151,
        (12494.50, np.inf):0.306087129,
        (-1, -0.99):-0.269949948,
        'other':-0.281391163
    },
    'at01s': {
        (-np.inf, 4.50):-0.405936416,
        (4.50, 6.50):-0.236264079,
        (6.50, 12.50):-0.092884308,
        (12.50, 14.50):-0.06817102,
        (14.50, 17.50):-0.012523543,
        (17.50, 30.50):0.072096111,
        (30.50, 35.50):0.150035011,
        (35.50, 46.50):0.211137797,
        (46.50, np.inf):0.310154524,
        'other':-0.405936416
    },
    'at104s': {
        (-np.inf, 16.50):0.190483424,
        (16.50, 30.50):0.111420475,
        (30.50, 37.50):0.079669463,
        (37.50, 49.50):0.009353816,
        (49.50, 63.50):-0.087887361,
        (63.50, 92.50):-0.321737218,
        (92.50, np.inf):-0.402222585,
        'other':0.190483424
    },
    'at32s': {
        (-np.inf, 931.50):-0.325572385,
        (931.50, 6095.50):-0.123705007,
        (6095.50, 10948.00):-0.119732623,
        (10948.00, 13331.50):-0.080223698,
        (13331.50, 20398.00):-0.045064621,
        (20398.00, 27826.00):0.064426263,
        (27826.00, 33893.50):0.069114138,
        (33893.50, 88015.50):0.162309991,
        (88015.50, np.inf):0.250504958,
        'other':-0.325572385
    },
    'atap01': {
        (-np.inf, 78.50):-0.520127366,
        (78.50, 194.50):-0.136698863,
        (194.50, 774.50):-0.034926067,
        (774.50, 880.50):-0.017850156,
        (880.50, 2278.00):0.049022194,
        (2278.00, 2909.00):0.173166884,
        (2909.00, np.inf):0.307856149,
        'other':-0.520127366
    },
    'au34s': {
        (0, 44.50):0.264502425,
        (44.50, 68.50):0.162944851,
        (68.50, 95.50):0.029958925,
        (95.50, np.inf):-0.044020048,
        (-3, -2.99):0.089678682,
        'other':-0.265602555
    },
    'aut222': {
        (0, 0.75):-0.270452414,
        (0.75, 0.99):0.019968301,
        (0.99, 1.02):0.160176909,
        (1.02, np.inf):0.084990305,
        (-1, -0.99):-0.264604828,
        'other':0.055833695
    },
    'balmag01': {
        (-np.inf, 133.50):0.235321781,
        (133.50, 168.50):0.349458738,
        (168.50, 192.50):0.154191256,
        (192.50, 220.50):0.04747031,
        (220.50, 292.50):0.022719271,
        (292.50, 376.50):-0.158047343,
        (376.50, np.inf):-0.256922331,
        'other':0.235321781
    },
    'bkc102': {
        (0, 3.50):0.020376707,
        (3.50, 5.50):0.029850524,
        (5.50, 8.50):0.131887294,
        (8.50, np.inf):0.030011412,
        (-1, -0.99):-0.278754368,
        'other':-0.126256641
    },
    'cvpl': {
        (-np.inf, 465.50):-0.506047701,
        (465.50, 481.50):-0.377614585,
        (481.50, 490.50):-0.326599725,
        (490.50, 503.50):-0.184969144,
        (503.50, 517.50):-0.024171368,
        (517.50, 533.50):0.132652393,
        (533.50, 552.50):0.304035615,
        (552.50, 568.50):0.403613528,
        (568.50, np.inf):0.764646469,
        'other':-0.506047701
    },
    'fi06s': {
        (0, 0.50):0.102122524,
        (0.50, 1.50):0.07810289,
        (1.50, 2.50):0.015636631,
        (2.50, 3.50):-0.161019625,
        (3.50, np.inf):-0.087218806,
        'other':-0.289811208
    },
    'fi20s': {
        (0, 20.50):-0.256459008,
        (20.50, 33.50):-0.127931478,
        (33.50, 74.50):0.091693656,
        (74.50, 128.50):0.139789355,
        (128.50, 146.50):0.15023869,
        (146.50, np.inf):0.275615989,
        'other':-0.289811208
    },
    'fi24s': {
        (0, 1.50):0.004301011,
        (1.50, 2.50):0.192845091,
        (2.50, np.inf):0.223767996,
        (-1, -0.99):-0.288604522,
        (-3, -2.99):0.058184603,
        'other':-0.087131108
    },
    'fi28s': {
        (0, 1771.00):-0.160733375,
        (1771.00, 20772.00):-0.014683041,
        (20772.00, 29782.50):0.048752656,
        (29782.50, 37246.00):0.080509165,
        (37246.00, 56047.00):0.190525083,
        (56047.00, np.inf):0.303127199,
        (-3, -2.99):0.058184603,
        'other':-0.288515032
    },
    'g102s': {
        (-np.inf, 0.50):-0.128645246,
        (0.50, 1.50):-0.031396182,
        (1.50, 3.50):0.053273663,
        (3.50, 4.50):0.065287115,
        (4.50, 9.50):0.087950073,
        (9.50, np.inf):0.173470714,
        'other':-0.128645246
    },
    'g207s': {
        (-np.inf, 81.50):-0.472549581,
        (81.50, 182.50):-0.111950482,
        (182.50, 594.50):-0.048594641,
        (594.50, 965.50):-0.011534922,
        (965.50, 1879.50):0.050356852,
        (1879.50, 2909.00):0.126490482,
        (2909.00, np.inf):0.392968358,
        'other':-0.472549581
    },
    'g404s': {
        (0, 0.50):0.07831223,
        (0.50, 1.50):0.000294842,
        (1.50, 2.50):-0.033237168,
        (2.50, np.inf):-0.285159667,
        (-1, -0.99):-0.011106379,
        'other':0.07831223
    },
    'linkc033': {
        (-np.inf, 1.00):0.018108757,
        (1.00, np.inf):0.13142863,
        'other':-0.171034365
    },
    'linkf036': {
        (-np.inf, 0.50):0.104642141,
        (0.50, 1.50):-0.082165327,
        (1.50, np.inf):-0.289812381,
        'other':-0.108773749
    },
    'linkf063': {
        (-np.inf, 1.00):0.042506323,
        (1.00, np.inf):-0.157866622,
        'other':-0.108773749
    },
    'linkf084': {
        (-np.inf, 0.50):0.120524933,
        (0.50, 1.50):-0.006998499,
        (1.50, 3.50):-0.067881909,
        (3.50, np.inf):-0.321655931,
        'other':-0.108773749
    },
    'linkf106': {
        (-np.inf, 730.50):-0.246885978,
        (730.50, 1151.50):-0.052093179,
        (1151.50, 1553.50):-0.022094153,
        (1553.50, 1884.50):0.225655836,
        (1884.50, np.inf):0.096474207,
        'other':-0.021180938
    },
    'linkf109': {
        (-np.inf, 0.50):0.120524933,
        (0.50, 1.50):-0.00323201,
        (1.50, 3.50):-0.107624504,
        (3.50, np.inf):-0.333609118,
        'other':-0.108773749
    },
    'linkf111': {
        (-np.inf, 0.50):0.11559454,
        (0.50, 1.50):0.067928747,
        (1.50, 3.50):0.065229616,
        (3.50, 5.50):-0.105069435,
        (5.50, np.inf):-0.318375005,
        'other':-0.108773749
    },
    'linkf148': {
        (-np.inf, 1.00):0.099748263,
        (1.00, np.inf):-0.196612463,
        'other':-0.108773749
    },
    'linkf151': {
        (-np.inf, 1.00):0.116471569,
        (1.00, np.inf):-0.096196091,
        'other':-0.108773749
    },
    'linkf182': {
        (-np.inf, np.inf):-0.373276183,
        'other':0.014929039
    },
    'linkf185': {
        (-np.inf, 2.50):-0.097620952,
        (2.50, 11.50):-0.22254267,
        (11.50, 26.50):-0.295196504,
        (26.50, 53.50):0.01647573,
        (53.50, 1408.50):0.122000539,
        (1408.50, np.inf):0.288212421,
        'other':-0.10176697
    },
    'linkf189': {
        (-np.inf, 70.50):-0.372223736,
        (70.50, 410.50):0.009164117,
        (410.50, 763.50):0.271585756,
        (763.50, 1506.50):0.005192876,
        (1506.50, np.inf):-0.005424265,
        'other':0.006265264
    },
    'm1150_stl_ft_tu': {
        (-np.inf, 686.50):-0.833836074,
        (686.50, 696.50):-0.661264393,
        (696.50, 708.50):-0.434930926,
        (708.50, 713.50):-0.251474954,
        (713.50, 725.50):-0.17950916,
        (725.50, 735.50):-0.030798204,
        (735.50, 742.50):0.124258331,
        (742.50, 764.50):0.272946615,
        (764.50, 776.50):0.380294706,
        (776.50, 791.50):0.468659734,
        (791.50, np.inf):0.633104882,
        'other':-0.833836074
    },
    'm1151_ins_ft_tu': {
        (-np.inf, 620.50):-0.460884142,
        (620.50, 633.50):-0.254588448,
        (633.50, 642.50):-0.205064579,
        (642.50, 654.50):-0.107378822,
        (654.50, 667.50):-0.04812492,
        (667.50, 688.50):0.056244559,
        (688.50, 707.50):0.094225593,
        (707.50, 722.50):0.198395013,
        (722.50, 738.50):0.427297559,
        (738.50, np.inf):0.605875061,
        'other':-0.460884142
    },
    'mtap01': {
        (0, 696.50):0.064536588,
        (696.50, 1205.00):0.357386316,
        (1205.00, np.inf):0.261871035,
        (-3, -2.99):0.178474994,
        'other':-0.069698151
    },
    'paymnt11': {
        (-np.inf, 4.50):-0.526907398,
        (4.50, 9.50):-0.351012335,
        (9.50, 21.50):-0.228338345,
        (21.50, 27.50):-0.013642907,
        (27.50, 34.50):0.032075707,
        (34.50, 45.50):0.039728767,
        (45.50, 82.50):0.130014134,
        (82.50, 144.50):0.212446747,
        (144.50, np.inf):0.516203291,
        'other':-0.526907398
    },
    'per233': {
        (0, 1.50):0.012406748,
        (1.50, 44.50):0.219640055,
        (44.50, 126.50):0.109367381,
        (126.50, np.inf):0.095283039,
        (-1, -0.99):-0.180131763,
        'other':0.072888338
    },
    'ret225': {
        (0, 1.51):0.058647123,
        (1.51, 2.82):0.137838183,
        (2.82, 4.82):0.20392408,
        (4.82, np.inf):0.006592964,
        (-1, -0.99):-0.239910468,
        'other':0.011338192
    },
    'rev205': {
        (-np.inf, 1.16):0.055317918,
        (1.16, 3.64):0.066268944,
        (3.64, 4.64):0.201240382,
        (4.64, 8.10):0.057506733,
        (8.10, np.inf):-0.217080382,
        'other':-0.347426404
    },
    's064a': {
        (-np.inf, 1462.00):-0.066353026,
        (1462.00, 3486.50):-0.03982915,
        (3486.50, 4725.00):0.123160703,
        (4725.00, 7652.00):0.002693967,
        (7652.00, np.inf):-0.124729064,
        'other':0.072815821
    },
    'score_cv_earlypaydefault': {
        (-np.inf, 543.50):-0.662443588,
        (543.50, 558.50):-0.396670503,
        (558.50, 570.50):-0.308230113,
        (570.50, 579.50):-0.211529547,
        (579.50, 599.50):-0.17014087,
        (599.50, 628.50):0.089057495,
        (628.50, 637.50):0.237729095,
        (637.50, 668.50):0.27959811,
        (668.50, np.inf):0.644759806,
        'other':-0.662443588
    },
    'score_cv_newacct_2': {
        (-np.inf, 550.50):-0.579470891,
        (550.50, 558.50):-0.344328607,
        (558.50, 567.50):-0.28139912,
        (567.50, 575.50):-0.190843341,
        (575.50, 580.50):-0.012154811,
        (580.50, 590.50):0.047985432,
        (590.50, 601.50):0.048569497,
        (601.50, 608.50):0.148961349,
        (608.50, 623.50):0.186152575,
        (623.50, np.inf):0.540543017,
        'other':-0.579470891
    },
    'scr_alt_b_2': {
        (-np.inf, 578.50):0.017227685,
        (578.50, 581.50):-0.14037727,
        (581.50, 584.50):-0.250936519,
        (584.50, 602.50):-0.119934169,
        (602.50, 611.50):0.049352486,
        (611.50, 623.50):0.100784787,
        (623.50, 635.50):0.28085187,
        (635.50, np.inf):0.504774596,
        'other':0.017227685
    },
    'scr_link_b_2': {
        (-np.inf, 539.50):-0.614448455,
        (539.50, 549.50):-0.287360296,
        (549.50, 561.50):-0.268199799,
        (561.50, 568.50):-0.186854159,
        (568.50, 576.50):-0.144139195,
        (576.50, 587.50):-0.00513191,
        (587.50, 591.50):-0.004522719,
        (591.50, 606.50):0.132222897,
        (606.50, 616.50):0.245981533,
        (616.50, 627.50):0.34519359,
        (627.50, np.inf):0.605131148,
        'other':-0.614448455
    },
    'se09s': {
        (0, 0.50):0.106823325,
        (0.50, 2.50):-0.011781127,
        (2.50, np.inf):-0.128977303,
        (-1, -0.99):-0.017286981,
        'other':0.106823325
    },
    'se20s': {
        (0, 10.50):-0.290879489,
        (10.50, 29.50):-0.173002338,
        (29.50, 47.50):0.02540852,
        (47.50, 87.50):0.120332347,
        (87.50, 122.50):0.123992689,
        (122.50, np.inf):0.243244829,
        (-1, -0.99):-0.017286981,
        'other':-0.290879489
    },
    'se34s': {
        (0, 72.50):0.141179209,
        (72.50, 91.50):-0.000708778,
        (91.50, 97.50):-0.158164803,
        (97.50, np.inf):-0.070654247,
        (-1, -0.99):-0.017286981,
        (-3, -2.99):0.049700215,
        'other':0.141179209
    },
    'vantagescore4': {
        (-np.inf, 493.50):-0.741871066,
        (493.50, 521.50):-0.500781429,
        (521.50, 533.50):-0.269735504,
        (533.50, 541.50):-0.20559178,
        (541.50, 549.50):-0.15303063,
        (549.50, 564.50):-0.089678042,
        (564.50, 581.50):0.051189134,
        (581.50, 604.50):0.229921676,
        (604.50, 615.50):0.328771776,
        (615.50, 628.50):0.438933624,
        (628.50, np.inf):0.659892197,
        'other':-0.741871066
    }
}

In [15]:
def map_woe(row, variable, ref_table):
    value = row[variable]
    woe_other = ref_table[variable].get('other', None)
    
    if pd.isna(value):
        return woe_other

    for bin, woe in ref_table[variable].items():
        if bin == 'other':
            continue
        elif isinstance(bin, tuple):
            if value >= bin[0] and value < bin[1]:
                return woe
        else:
            if value == bin:
                return woe
    
    return woe_other

db_6 = db_5.copy()

# Map WoE values for each variable
for variable in db_6.columns:
    if variable not in ref_table:
        continue

    # Create the WoE column
    db_6[f'{variable}_WoE'] = db_6.apply(lambda row: map_woe(row, variable, ref_table), axis=1)

    # Rename the original column
    db_6.rename(columns={variable: f'{variable}_Orig'}, inplace=True)


In [ ]:
db_6.to_csv('optbinning_result_woe_new_and_returning(denied).csv', index=False)